<a href="https://colab.research.google.com/github/Tariik7/BIGDATALABS/blob/main/spark1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-binhadoop3.2.tgz

!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
!pip install -q pymongo matp

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,633 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,565 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Get:1

In [ ]:
import os
import sys
import findspark
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
findspark.init()
findspark.find()

'/usr/local/lib/python3.12/dist-packages/pyspark'

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName("ColabSpark") \
.config("spark.driver.memory", "2g") \
.getOrCreate()
print(" Spark est configuré avec succès !")

 Spark est configuré avec succès !


In [ ]:
print(spark.version)
data = [(1, "Alice", 23), (2, "Bob", 30), (3, "Charlie", 29)]
columns = ["id", "nom", "age"]
df = spark.createDataFrame(data, columns)
df.show()
df.printSchema() # Structure du DataFrame
df.select("nom", "age").show() # Sélection de colonnes
df.filter(df.age > 25).show() # Filtrage des données

4.0.1
+---+-------+---+
| id|    nom|age|
+---+-------+---+
|  1|  Alice| 23|
|  2|    Bob| 30|
|  3|Charlie| 29|
+---+-------+---+

root
 |-- id: long (nullable = true)
 |-- nom: string (nullable = true)
 |-- age: long (nullable = true)

+-------+---+
|    nom|age|
+-------+---+
|  Alice| 23|
|    Bob| 30|
|Charlie| 29|
+-------+---+

+---+-------+---+
| id|    nom|age|
+---+-------+---+
|  2|    Bob| 30|
|  3|Charlie| 29|
+---+-------+---+



In [ ]:
#On va charger un dataset CSV et l’analyser avec Spark
df = spark.read.csv("/content/sample_data/transaction_data.csv", header=True, inferSchema=True)

df.show(5)
#▪ Afficher le schéma des données
df.printSchema()
# Filtrer les transactions supérieures à 1000
df.filter(df["Transaction Amount"] > 1000).show()
#Calculer le montant total des transactions par type
df.groupBy("Transaction Type").sum("Transaction Amount").show()
#Trier les transactions par montant décroissant
df.orderBy(df["Transaction Amount"].desc()).show(5)

+--------------+-----------------+-------------------+------------------+----------------+-------------------+------------------+----------+--------------------------------+-----------+----------------+------------+----------------------+--------+
|Transaction ID|Sender Account ID|Receiver Account ID|Transaction Amount|Transaction Type|          Timestamp|Transaction Status|Fraud Flag|Geolocation (Latitude/Longitude)|Device Used|Network Slice ID|Latency (ms)|Slice Bandwidth (Mbps)|PIN Code|
+--------------+-----------------+-------------------+------------------+----------------+-------------------+------------------+----------+--------------------------------+-----------+----------------+------------+----------------------+--------+
| TXN9520068950|         ACC14994|           ACC16656|             495.9|         Deposit|2025-01-17 10:14:00|            Failed|      true|            34.0522 N, -74.006 W|    Desktop|          Slice3|          10|                   179|    3075|
| TXN941